# Multi-Agent Tennis

## The Problem
In this task, two reinforcement-learning agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1. If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01. The goal of each agent is to keep the ball in play.

### The Environment
We are using the Tennis environment from the Unity ML-Agents plugin.

The observation space consists of 24 variables corresponding to the position and velocity of the ball and racket. Each agent receives its own, local observation. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. After each episode, we add up the rewards that each agent received (without discounting), to get a score for each agent. This yields 2 (potentially different) scores. We then take the maximum of these 2 scores as the episode score. The environment is considered solved when the average episode score is at least +0.5 over 100 consecutive episodes.

### Set Up
Make sure [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) is installed. Then, run the following command to install the required packages.

In [ ]:
!pip install -r requirements.txt

Next, we import all necessary libraries.

In [ ]:
from collections import deque, namedtuple
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from unityagents import UnityEnvironment

To load the Unity environment, change the *file\_name* parameter based on your operating system, as specified below:
* Windows (x86): "envs/Tennis_Windows_x86/Tennis.exe"
* Windows (x86_64): "envs/Tennis_Windows_x86_64/Tennis.exe"* 
* Linux (x86): "envs/Tennis_Linux/Tennis.x86"
* Linux (x86_64): "envs/Tennis_Linux/Tennis.x86_64"
* Mac: "envs/Tennis.app"

For example,
```
env = UnityEnvironment(file_name="envs/Tennis_Linux/Tennis.x86_64")
```

In [ ]:
env = UnityEnvironment(file_name="envs/Tennis_Linux/Tennis.x86_64")

To examine the environment in more detail, run the following code chunk. As can be seen, there are 2 agents, each state consists of 24 components, and each action has 2 components.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

We can now construct the agent.

## The Solution: A Multi-Agent Deep Deterministic Policy Gradient (MADDPG) Approach

### Actor-Critic Networks with Batch Normalization
To implement DDPG, we use an Actor-Critic network structure shown below, similar to the one suggested by the [DDPG paper (Lillicrap et al, 2015)](https://arxiv.org/pdf/1509.02971.pdf), from which we implemented the described network structure but with the addition of batch normalization layers and some changes to the hyperparameters. The addition of batch normalization layers dramatically increased the agents' performance by normalizing the weights of the networks' hidden layers, thus reducing covariance shift and boosting the stability of the network while reducing the training time. Each agent has its own actor network, which takes in that agent's view of the state and outputs an action for that agent to take. The critic network takes in a (state, action) pair and outputs a corresponding estimated value. The critic network is shared by all agents to take into consideration the interaction between the agents and how that affects the resulting reward. We found that using hidden layer dimensions of 512 and 256 worked better than 400 and 300, which the network in the paper uses.

In [ ]:
def fan_in(size):  # helper method for initializing weights
    f = size[0]
    bound = 1.0 / np.sqrt(f)
    return torch.Tensor(size).uniform_(-bound, bound)

class Actor(nn.Module):
    def __init__(self, state_size, action_size, hidden1=512, hidden2=256, final_weights_init=3e-3):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.final_weights_init = final_weights_init
        self.fc1 = nn.Linear(self.state_size, hidden1)
        self.bn1 = nn.BatchNorm1d(hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.bn2 = nn.BatchNorm1d(hidden2)
        self.fc3 = nn.Linear(hidden2, self.action_size)
        self.init_weights()
        
    def init_weights(self):
        self.fc1.weight.data = fan_in(self.fc1.weight.data.size())
        self.fc2.weight.data = fan_in(self.fc2.weight.data.size())
        self.fc3.weight.data.uniform_(-self.final_weights_init, self.final_weights_init)
        
    def forward(self, state):
        x = self.bn1(F.relu(self.fc1(state)))
        x = self.bn2(F.relu(self.fc2(x)))
        return F.tanh(self.fc3(x))

class Critic(nn.Module):
    def __init__(self, state_size, action_size, hidden1=512, hidden2=256, final_weights_init=3e-3):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.num_agents = num_agents
        self.final_weights_init = final_weights_init
        self.fc1 = nn.Linear(self.state_size, hidden1)
        self.bn1 = nn.BatchNorm1d(hidden1)
        self.fc2 = nn.Linear(hidden1+self.action_size, hidden2)
        self.fc3 = nn.Linear(hidden2, 1)
        self.init_weights()
        
    def init_weights(self):
        self.fc1.weight.data = fan_in(self.fc1.weight.data.size())
        self.fc2.weight.data = fan_in(self.fc2.weight.data.size())
        self.fc3.weight.data.uniform_(-self.final_weights_init, self.final_weights_init)
        
    def forward(self, state_action):
        state, action = state_action
        x = self.bn1(F.relu(self.fc1(state)))
        x = F.relu(self.fc2(torch.cat([x, action], 1)))
        return self.fc3(x)

### A Memory Buffer for Experience Replay
We use a ReplayBuffer, defined below, to implement experience replay in order to sequentially decorrelate the observations and possibly reuse training data. Each agent has its own ReplayBuffer, which stores observations at each time step. To update their own actor networks and the shared critic network, each agent samples randomly from this buffer to extract minibatches of (state, action, reward, next_state, done) tuples.

In [ ]:
class ReplayBuffer():
    def __init__(self, capacity, minibatch_size, device):
        self.buffer = deque(maxlen=capacity)
        self.experience = namedtuple("Experience", field_names=['states', 'actions', 'rewards', 'next_states', 'dones'])
        self.device = device
        self.minibatch_size = minibatch_size
        self.length = 0
        
    def add(self, s, a, r, n_s, d):
        self.buffer.append(self.experience(s, a, r, n_s, d))
        self.length += 1
        
    def sample(self):
        exp = random.sample(self.buffer, k=self.minibatch_size)
        states = torch.from_numpy(np.vstack([e.states for e in exp if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e.actions for e in exp if e is not None])).float().to(self.device)
        rewards = torch.from_numpy(np.vstack([e.rewards for e in exp if e is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e.next_states for e in exp if e is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e.dones for e in exp if e is not None])).float().to(self.device)
        return (states, actions, rewards, next_states, dones)
    
    def get_len(self):
        return self.length

I made an attempt to implement Prioritized Experience Replay with a Binary Sum Tree (where the priority of each experience tuple was stored in the leaf nodes and each parent node is the sum of the priority of its children), but the performance of the agents using this structure was nowhere near that with the vanilla replay deque. The code for the prioritized experience replay buffer is in ```prioritized_exp_replay.py```, and the corresponding _Agent_ and _MADDPG_ classes are in ```per_agent.py```.

### Ornstein-Uhlenbeck Random Noise for Action Exploration
The DDPG paper makes use of the Ornstein-Uhlenbeck Process for generating random noise for momentum-based processes. This noise will encourage the agent to explore new actions, and the magnitude of the noise decreases over time as the variance decays with each episode, down to a minimum value of 0.05, where it remains for the remainder of training. For this task, we use parameters of mu=0, theta=0.15, sigma=0.1, and sigma_decay=0.98.

In [ ]:
class OrnsteinUhlenbeck():
    def __init__(self, num_agents, action_size, mu=0, theta=0.15, sigma=0.1, sigma_min=0.05, sigma_decay=0.98):
        self.num_agents = num_agents
        self.action_size = action_size
        self.mu = mu
        self.sigma = sigma
        self.sigma_min = sigma_min
        self.sigma_decay = sigma_decay
        self.theta = theta
        self.prev_val = np.zeros((1, self.action_size))
    
    def sample(self):
        val = self.prev_val + self.theta * (self.mu - self.prev_val) * self.sigma * np.random.normal(size=(1, self.action_size))
        self.prev_val = val
        return val
    
    def reset(self):
        self.prev_val = np.zeros((1, self.action_size))
        self.sigma = max(self.sigma_min, self.sigma*self.sigma_decay)

### The Agent
The Agent class is shown below. As discussed above, each agent contains its own actor network and replay buffer, but the critic network is shared by all agents. The learning rate for the actor networks is set to 1e-3. The capacity of the replay buffer is 1e6, and the minibatch size is 256, which seems to work well.

In [ ]:
class Agent():
    def __init__(self, state_size, action_size, device, buffer_capacity, minibatch_size, lr_actor=1e-3):
        self.state_size = state_size
        self.action_size = action_size
        self.device = device
        self.buffer = ReplayBuffer(int(buffer_capacity), minibatch_size, self.device)
        self.noise = OrnsteinUhlenbeck(num_agents=num_agents, action_size=action_size)
        
        self.actor_local = Actor(state_size, action_size).to(self.device)
        self.actor_target = Actor(state_size, action_size).to(self.device)
        self.copy_weights(self.actor_target, self.actor_local)
        self.optim_actor = optim.Adam(self.actor_local.parameters(), lr=lr_actor)    
    
    def copy_weights(self, target_network, source_network):
        for target_param, param_to_copy in zip(target_network.parameters(), source_network.parameters()):
            target_param.data.copy_(param_to_copy.data)
    
    def reset(self):
        self.noise.reset()
    
    def select_action(self, state):  # states is shape (1, state_size)
        state = torch.from_numpy(state).unsqueeze(0).float().to(self.device)
        self.actor_local.eval()
        with torch.no_grad():
            action = self.actor_local(state)  # shape (1, action_size)
        self.actor_local.train()
        return np.clip(action.cpu().data.numpy() + self.noise.sample(), -1, 1)
        
    def add_to_buffer(self, state, action, reward, next_state, done):
        self.buffer.add(state, action, reward, next_state, done)
    
    def get_buffer_samples(self):
        return self.buffer.sample()

### The Multi-Agent Framework
Shown below is the class that acts as the directly interface between the environment and the agents. It contains both agents, as well as the shared critic networks that use a learning rate of 1e-3. For both the actor and critic, two copies of each network are used; the target network is updated more slowly and is used to compute the TD target, parametrized by gamma=0.99, in the TD error when updating the local critic network. Both target networks are updated using a soft update, parametrized by tau=0.99, which gradually combines the target networks' existing weights with those of the local networks. The actor and critic networks are updated at every time step, though the code supports updating every update\_freq timesteps. Since the updating of each agent's local actor network requires input from the critic network, the actor network updates are also taken care of in this class.

In [ ]:
class MADDPG():
    def __init__(self, state_size, action_size, num_agents, buffer_capacity=1e6, minibatch_size=256, update_freq=1, tau=1e-3, gamma=0.99, lr_critic=1e-3):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.state_size = state_size
        self.action_size = action_size
        self.num_agents = num_agents
        self.agents = [Agent(state_size, action_size, self.device, buffer_capacity, minibatch_size) for i in range(num_agents)]
        
        self.shared_critic_local = Critic(state_size, action_size).to(self.device)
        self.shared_critic_target = Critic(state_size, action_size).to(self.device)
        self.optim_critic = optim.Adam(self.shared_critic_local.parameters(), lr=lr_critic)
        self.copy_weights(self.shared_critic_target, self.shared_critic_local)
        
        self.minibatch_size = minibatch_size
        self.update_freq = update_freq
        self.tau = tau
        self.gamma = gamma
        self.num_steps = 0
    
    def copy_weights(self, target_network, source_network):
        for target_param, param_to_copy in zip(target_network.parameters(), source_network.parameters()):
            target_param.data.copy_(param_to_copy.data)
            
    def get_actions(self, states):
        actions = [self.agents[i].select_action(states[i]) for i in range(self.num_agents)]
        return np.concatenate(actions)
    
    def reset(self):
        for i in range(self.num_agents):
            self.agents[i].reset()
    
    def update(self, states, actions, rewards, next_states, dones):
        for i in range(self.num_agents):
            done = 1 if dones[i] else 0
            self.agents[i].add_to_buffer(states[i], actions[i], rewards[i], next_states[i], done)
        
        self.num_steps = (self.num_steps + 1) % self.update_freq
        if self.num_steps == 0 and self.agents[0].buffer.get_len() >= self.minibatch_size:
            self.update_local()
            
    def update_local(self):
        for i in range(self.num_agents):
            s, a, r, n_s, d = self.agents[i].get_buffer_samples()
            
            # update actor
            actions_pred = self.agents[i].actor_local(s)
            actor_loss = -self.shared_critic_local((s, actions_pred)).mean()
            self.agents[i].optim_actor.zero_grad()
            actor_loss.backward()
            self.agents[i].optim_actor.step()

            # update critic
            next_actions_target = self.agents[i].actor_target(n_s)
            future_rewards_target = self.shared_critic_target((n_s, next_actions_target))
            value_target = r + self.gamma*future_rewards_target*(1-d)
            value_local = self.shared_critic_local((s, a))
            critic_loss = F.mse_loss(value_local, value_target)
            self.optim_critic.zero_grad()
            critic_loss.backward()
            nn.utils.clip_grad_norm_(self.shared_critic_local.parameters(), 1)
            self.optim_critic.step()

            self.soft_update(self.agents[i].actor_target, self.agents[i].actor_local)
        self.soft_update(self.shared_critic_target, self.shared_critic_local)
            
    def soft_update(self, target_network, local_network):
        for target_param, local_param in zip(target_network.parameters(), local_network.parameters()):
            target_param.data.copy_(self.tau*local_param.data + (1-self.tau)*target_param.data)

### Putting it Together
Here, we train the agents. We use a cap of 5000 episodes, and allow each episode to play out for a maximum of 1000 timesteps. At the end of each episode, the high score of the two agents is stored into the buffer *score\_window*, and the task is completed when the average of the values within *score\_window* reaches or exceeds +0.5. The actor models are saved into *actor0.pth* and *actor1.pth* (corresponding to each agent), and the shared critic model is saved into *critic.pth*. The learning curve is then plotted to show the episode score throughout training, and is saved into *learning\_curve.png*.

In [ ]:
# reset environment
env_info = env.reset(train_mode=True)[brain_name]
states = env_info.vector_observations
num_agents, state_size = states.shape
action_size = brain.vector_action_space_size
max_timesteps = 1000

num_episodes = 5000

agents = MADDPG(state_size=state_size, action_size=action_size, num_agents=num_agents)
score_window = deque(maxlen=100)
score_record = []
highest_score = 0

print(f'Training starts with {num_agents} agents!')
for ep in range(1, num_episodes+1):
    scores = np.zeros(num_agents)
    agents.reset()
    env_info = env.reset(train_mode=True)[brain_name]
    states = env_info.vector_observations
    for step in range(max_timesteps):
        actions = agents.get_actions(states)
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        scores += rewards
        dones = env_info.local_done
        agents.update(states, actions, rewards, next_states, dones)
        states = next_states
        if np.any(dones):
            break
    max_score = np.max(scores)
    score_window.append(max_score)
    score_record.append(max_score)
    avg_score = np.mean(score_window)
    if max_score > highest_score:
        highest_score = max_score
    print(f'\rEpisode {ep}/{num_episodes}: Higher Score = {max_score}, Highest Score = {highest_score}, Avg = {avg_score}', end="")
    if avg_score >= 0.5:
        print(f'\nEnvironment solved in {ep} episodes!   Average Score: {avg_score}')
        break
        
# save the trained model
torch.save(agents.agents[0].actor_local.state_dict(), 'actor0.pth')
torch.save(agents.agents[1].actor_local.state_dict(), 'actor1.pth')
torch.save(agents.shared_critic_local.state_dict(), 'critic.pth')

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(score_record)), score_record)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.savefig('learning_curve.png')

This task was solved in 556 episodes, and the learning curve is pictured below:
![learning curve](learning_curve.png)

At first, hits are very sparsely scattered, but by around episode 200, the agents can pretty consistently get one shot across the court between the two of them. By episode 350, a score of 0 is rare, and the agents more frequently reach scores of 0.2 and 0.3. By episode 475, the score skyrockets, reaching a maximum of 2.7 past 2.5 multiple times as the agents get the hang of the game. By episode 556, the average high score across 100 consecutive episodes as surpassed +0.5, and the task is solved.

After satisfactory performance is achieved, we can close the environment.

In [ ]:
env.close()

Future work involves further investigation into [prioritized experience replay (Schaul et al, 2016)](https://arxiv.org/pdf/1511.05952.pdf) and how to efficiently sample from and maintain the priorities of each experience tuple. I would be interested in exploring how the more agents would affect the training process, such as in a game of doubles rather than singles. Since this task is purely collaborative, I also want to explore how adding competition between the two agents would affect their performance (i.e. giving an agent an additional reward of +0.1 if they win the point after hitting the ball over the net).